In [6]:
import pandas as pd

raw_df = pd.read_csv("BRCA_mRNA_ merged.norm.csv").T

In [2]:
clinical_df = pd.read_csv("ClinicalData_BRCA_All_CDE.csv").T
clinical_df.columns = clinical_df.iloc[0]
clinical_df = clinical_df.iloc[pd.RangeIndex(len(clinical_df)).drop(0)]

clinical_df.reset_index(inplace=True)

In [3]:
clinical_df.head(2)

Column1,index,bcr_patient_barcode,bcr_patient_uuid,patient_id,vital_status,days_to_death,days_to_last_followup,additional_studies,additional_surgery_locoregional_procedure,additional_surgery_metastatic_procedure,...,therapy_type-2,tissue_prospective_collection_indicator,tissue_retrospective_collection_indicator,tissue_source_site,tumor_tissue_site,tumor_tissue_site_other,withdrawn,year_of_dcc_upload,year_of_form_completion,year_of_initial_pathologic_diagnosis
0,Column2,tcga-5l-aat0,86c6f993-327f-4525-9983-29c55625593a,aat0,alive,NaN,1477,NaN,NaN,NaN,...,NaN,no,yes,5l,breast,NaN,false,2016,2014,2010
1,Column3,tcga-5l-aat1,16fc3677-0393-4ed1-ad3f-c8355f056369,aat1,alive,NaN,1471,NaN,NaN,NaN,...,NaN,no,yes,5l,breast,NaN,false,2016,2014,2010


In [ ]:
#indexNames = clinical_df[clinical_df ["breast_carcinoma_estrogen_receptor_status"] == "NaN"].index
#clinical_df.drop(indexNames , inplace=True)

In [8]:
raw_df.columns = raw_df.iloc[0]
raw_df = raw_df.iloc[pd.RangeIndex(len(raw_df)).drop(0)]
#raw_df.reset_index(inplace=True)

In [22]:
raw_df.head(2)

Column1,index,gene_id,?|100130426,?|100133144,?|100134869,?|10357,?|10431,?|136542,?|155060,?|26823,...,ZXDB|158586,ZXDC|79364,ZYG11A|440590,ZYG11B|79699,ZYX|7791,ZZEF1|23140,ZZZ3|26009,psiTPTE22|387590,tAKR|389932,h_or_d
Column2,TCGA-3C-AAAU-01A-11R-A41B-07,normalized_count,0.0000,16.3644,12.9316,52.1503,408.0760,0.0000,1187.0050,0.0000,...,1007.78,1658.5,258.494,1208.37,3507.25,1894.93,1180.46,1.7233,0,Diseased
Column3,TCGA-3C-AALI-01A-11R-A41B-07,normalized_count,0.0000,9.2659,17.3790,69.7553,563.8934,0.0000,516.0413,1.0875,...,448.613,1343.12,198.477,603.589,5504.62,1318.65,406.743,926.591,0,Diseased


In [10]:
raw_df = raw_df.rename(columns = {"Hybridization REF":"index"})

In [18]:
#function to get patient id
def patient_id (sample_id):
    return str.lower(sample_id[:12])

#function to classify as healthy or diseased, get last two characters of sample id  
def h_or_d (sample_id):
    x = sample_id[13:15]
    if x == "11":
        return "Healthy"
    else:
        return "Diseased"

In [19]:
raw_df["h_or_d"] = raw_df.apply(lambda x : h_or_d(x["index"]),axis=1) # Column to store if healthy or dis

In [20]:
raw_df["h_or_d"]

Column2       Diseased
Column3       Diseased
Column4       Diseased
Column5       Diseased
Column6       Diseased
                ...   
Column1209    Diseased
Column1210    Diseased
Column1211    Diseased
Column1212    Diseased
Column1213    Diseased
Name: h_or_d, Length: 1212, dtype: object

In [23]:
#function to classify as Triple negative or non-TNBC or NA
def triple_negative (er, pr, her2):
    if er == "negative" and pr == "negative" and her2 == "negative":
        return "TNBC"
    elif er == "positive" or pr == "positive" or her2 == "positive":
        return "non-TNBC"
    else:
        return float ("NaN")

In [24]:
# Column to store if TNBC or non-TNBC or NA
clinical_df["cancer_class"] = clinical_df.apply(lambda x : \
triple_negative(x["breast_carcinoma_estrogen_receptor_status"], x["breast_carcinoma_progesterone_receptor_status"], \
                x["lab_proc_her2_neu_immunohistochemistry_receptor_status"]),axis=1) 

In [25]:
clinical_df["cancer_class"]

0       non-TNBC
1       non-TNBC
2           TNBC
3       non-TNBC
4       non-TNBC
          ...   
1092    non-TNBC
1093    non-TNBC
1094    non-TNBC
1095    non-TNBC
1096    non-TNBC
Name: cancer_class, Length: 1097, dtype: object

In [26]:
raw_df["patient_id"] = raw_df.apply(lambda x : patient_id(x["index"]),axis=1) # Column to store if healthy or dis

In [27]:
classified_df = pd.merge(raw_df, clinical_df[["bcr_patient_barcode", "cancer_class"]], left_on = "patient_id", right_on = "bcr_patient_barcode", how = "inner")
#merge clinical data to RNAseq data

In [28]:
classified_df.head(2)

Column1,index,gene_id,?|100130426,?|100133144,?|100134869,?|10357,?|10431,?|136542,?|155060,?|26823,...,ZYG11B|79699,ZYX|7791,ZZEF1|23140,ZZZ3|26009,psiTPTE22|387590,tAKR|389932,h_or_d,patient_id,bcr_patient_barcode,cancer_class
0,TCGA-3C-AAAU-01A-11R-A41B-07,normalized_count,0.0000,16.3644,12.9316,52.1503,408.0760,0.0000,1187.0050,0.0000,...,1208.37,3507.25,1894.93,1180.46,1.7233,0,Diseased,tcga-3c-aaau,tcga-3c-aaau,non-TNBC
1,TCGA-3C-AALI-01A-11R-A41B-07,normalized_count,0.0000,9.2659,17.3790,69.7553,563.8934,0.0000,516.0413,1.0875,...,603.589,5504.62,1318.65,406.743,926.591,0,Diseased,tcga-3c-aali,tcga-3c-aali,non-TNBC


In [29]:
classified_df ["Class"] = classified_df ["cancer_class"]

In [30]:
classified_df.loc [classified_df["h_or_d"] == "Healthy", "Class"] = "Healthy"

In [38]:
final_classified_df = classified_df.drop(['h_or_d', "patient_id", "bcr_patient_barcode", "cancer_class", "gene_id"], axis=1)
# drop unneeded columns
final_classified_df

Column1,index,?|100130426,?|100133144,?|100134869,?|10357,?|10431,?|136542,?|155060,?|26823,?|280660,...,ZXDB|158586,ZXDC|79364,ZYG11A|440590,ZYG11B|79699,ZYX|7791,ZZEF1|23140,ZZZ3|26009,psiTPTE22|387590,tAKR|389932,Class
0,TCGA-3C-AAAU-01A-11R-A41B-07,0.0000,16.3644,12.9316,52.1503,408.0760,0.0000,1187.0050,0.0000,0.0000,...,1007.78,1658.5,258.494,1208.37,3507.25,1894.93,1180.46,1.7233,0,non-TNBC
1,TCGA-3C-AALI-01A-11R-A41B-07,0.0000,9.2659,17.3790,69.7553,563.8934,0.0000,516.0413,1.0875,0.5438,...,448.613,1343.12,198.477,603.589,5504.62,1318.65,406.743,926.591,0,non-TNBC
2,TCGA-3C-AALJ-01A-31R-A41B-07,0.9066,11.6228,9.2294,154.2974,1360.8341,0.0000,592.0218,0.0000,0.0000,...,533.998,768.812,331.822,532.185,5458.75,942.883,509.519,35.3581,0,non-TNBC
3,TCGA-3C-AALK-01A-11R-A41B-07,0.0000,12.0894,11.0799,143.8643,865.5358,0.0000,552.7513,0.4137,0.0000,...,437.733,863.881,175.424,607.365,5691.35,781.134,700.869,66.6115,0,non-TNBC
4,TCGA-4H-AAAK-01A-12R-A41B-07,0.0000,6.8468,14.4298,84.2128,766.3830,0.0000,260.8511,0.4255,0.0000,...,424.255,1049.79,14.0426,775.745,4041.7,831.915,881.702,187.234,0,non-TNBC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1207,TCGA-WT-AB44-01A-11R-A41B-07,0.0000,0.0000,3.2580,42.2643,1877.4180,0.0000,704.5408,4.0725,0.0000,...,243.535,772.959,98.5543,315.211,10937.1,800.652,443.087,724.903,0,non-TNBC
1208,TCGA-XX-A899-01A-11R-A36F-07,0.0000,14.3858,21.4409,137.7756,652.7559,0.0000,427.1654,2.7559,0.0000,...,643.701,1266.54,21.2598,688.189,5118.11,1933.86,670.079,98.4252,0,non-TNBC
1209,TCGA-XX-A89A-01A-11R-A36F-07,0.0000,22.3240,27.2744,64.1427,722.7208,0.0000,376.4761,0.4724,0.0000,...,341.521,1375.53,164.384,746.812,5477.56,1437.41,953.708,235.238,0.9447,non-TNBC
1210,TCGA-Z7-A8R5-01A-42R-A41B-07,0.0000,2.2638,7.2933,85.0461,1140.2801,0.0000,252.0682,0.5973,0.0000,...,248.484,796.225,51.9667,505.928,6675.63,754.413,750.829,238.927,0,non-TNBC


In [39]:
# Move Class to second column
col_name="Class"
second_col = final_classified_df.pop(col_name)
final_classified_df.insert(1, col_name, second_col)
# Rename index as Sample-id
final_classified_df = final_classified_df.rename(columns = {"index":"Sample-id"})

In [40]:
final_classified_df = final_classified_df.dropna() # drop samples with NA values from dataset
final_classified_df.head()

Column1,Sample-id,Class,?|100130426,?|100133144,?|100134869,?|10357,?|10431,?|136542,?|155060,?|26823,...,ZXDA|7789,ZXDB|158586,ZXDC|79364,ZYG11A|440590,ZYG11B|79699,ZYX|7791,ZZEF1|23140,ZZZ3|26009,psiTPTE22|387590,tAKR|389932
0,TCGA-3C-AAAU-01A-11R-A41B-07,non-TNBC,0.0000,16.3644,12.9316,52.1503,408.0760,0.0000,1187.0050,0.0000,...,129.592,1007.78,1658.5,258.494,1208.37,3507.25,1894.93,1180.46,1.7233,0
1,TCGA-3C-AALI-01A-11R-A41B-07,non-TNBC,0.0000,9.2659,17.3790,69.7553,563.8934,0.0000,516.0413,1.0875,...,59.8151,448.613,1343.12,198.477,603.589,5504.62,1318.65,406.743,926.591,0
2,TCGA-3C-AALJ-01A-31R-A41B-07,non-TNBC,0.9066,11.6228,9.2294,154.2974,1360.8341,0.0000,592.0218,0.0000,...,35.3581,533.998,768.812,331.822,532.185,5458.75,942.883,509.519,35.3581,0
3,TCGA-3C-AALK-01A-11R-A41B-07,non-TNBC,0.0000,12.0894,11.0799,143.8643,865.5358,0.0000,552.7513,0.4137,...,55.0269,437.733,863.881,175.424,607.365,5691.35,781.134,700.869,66.6115,0
4,TCGA-4H-AAAK-01A-12R-A41B-07,non-TNBC,0.0000,6.8468,14.4298,84.2128,766.3830,0.0000,260.8511,0.4255,...,48.9362,424.255,1049.79,14.0426,775.745,4041.7,831.915,881.702,187.234,0


In [41]:
final_classified_df.loc[classified_df.Class == "non-TNBC", "Class"].count()

867

In [42]:
final_classified_df.loc[classified_df.Class == "TNBC", "Class"].count()

116

In [43]:
final_classified_df.loc[classified_df.Class == "Healthy", "Class"].count()

112

In [47]:
len(final_classified_df.index)

1095

In [44]:
final_classified_df.to_csv(path_or_buf = "Preprocessed_BRCA_RNA_data_normalized.csv", index = False) # write to csv